<a href="https://colab.research.google.com/github/yeonghwanchoi/EDA/blob/master/jeju_EDA/sale_trend_jeju.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Important library for many geopython libraries
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 
# Install Folium for Geographic data visualization
!pip install folium
# Install plotlyExpress
!pip install plotly_express

Reading package lists... Done
Building dependency tree       
Reading state information... Done
gdal-bin is already the newest version (2.2.3+dfsg-2).
python-gdal is already the newest version (2.2.3+dfsg-2).
The following additional packages will be installed:
  python3-numpy
Suggested packages:
  python-numpy-doc python3-nose python3-numpy-dbg
The following NEW packages will be installed:
  python3-gdal python3-numpy
0 upgraded, 2 newly installed, 0 to remove and 39 not upgraded.
Need to get 2,288 kB of archives.
After this operation, 13.2 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 python3-numpy amd64 1:1.13.3-2ubuntu1 [1,943 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python3-gdal amd64 2.2.3+dfsg-2 [346 kB]
Fetched 2,288 kB in 2s (956 kB/s)
Selecting previously unselected package python3-numpy.
(Reading database ... 160772 files and directories currently installed.)
Preparing to unpack .../python3-numpy_1%

In [2]:
# government aid to jeju island  

In [3]:
# 1. data preprocessing

In [4]:
#data load

from google.colab import drive
drive.mount('/content/drive')

import warnings
warnings.filterwarnings(action='ignore')

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

plt.rc('font', family='AppleGothic')

import folium 
import geopandas as gpd
from IPython.display import Image

import tqdm # progress bar 
import os, sys

from scipy.spatial import distance
from sklearn.cluster import AgglomerativeClustering



Mounted at /content/drive


In [5]:
path = '/content/drive/MyDrive/colab/jeju/data/'
data5= pd.read_csv(os.path.join(path,'KRI-DAC_Jeju_data5.txt'))
data6= pd.read_csv(os.path.join(path,'KRI-DAC_Jeju_data6.txt'))
data7= pd.read_csv(os.path.join(path,'KRI-DAC_Jeju_data7.txt'))
data8= pd.read_csv(os.path.join(path,'KRI-DAC_Jeju_data8.txt'))


df = pd.concat([data5, data6, data7, data8]).drop(['X','Y'], axis=1).reset_index(drop=True)
df.columns=df.columns.str.lower()

In [7]:
df.head()

,objectid,field1,ym,sido,sigungu,franclass,type,time,totalspent,disspent,numofspent,numofdisspent,point_x,point_y
0,1,1,202005,제주특별자치도,제주시,영세,일반한식,00시,363000,66500,10,2,877005.9834,1.479766e+06
1,2,2,202005,제주특별자치도,제주시,영세,단란주점,00시,1180000,0,3,0,877005.7447,1.479816e+06
2,3,3,202005,제주특별자치도,제주시,중소1,편의점,00시,157670,6850,20,2,877056.6756,1.479616e+06
3,4,4,202005,제주특별자치도,제주시,영세,편의점,00시,46600,0,2,0,877055.9593,1.479766e+06
4,5,5,202005,제주특별자치도,제주시,영세,주점,00시,66000,0,2,0,877055.4817,1.479866e+06


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119429 entries, 0 to 1119428
Data columns (total 14 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   objectid       1119429 non-null  int64  
 1   field1         1119429 non-null  int64  
 2   ym             1119429 non-null  int64  
 3   sido           1119429 non-null  object 
 4   sigungu        1119429 non-null  object 
 5   franclass      1119429 non-null  object 
 6   type           1119429 non-null  object 
 7   time           1119429 non-null  object 
 8   totalspent     1119429 non-null  int64  
 9   disspent       1119429 non-null  int64  
 10  numofspent     1119429 non-null  int64  
 11  numofdisspent  1119429 non-null  int64  
 12  point_x        1119429 non-null  float64
 13  point_y        1119429 non-null  float64
dtypes: float64(2), int64(7), object(5)
memory usage: 119.6+ MB


In [9]:
df.describe()

,objectid,field1,ym,totalspent,disspent,numofspent,numofdisspent,point_x,point_y
count,1.119429e+06,1.119429e+06,1.119429e+06,1.119429e+06,1.119429e+06,1.119429e+06,1.119429e+06,1.119429e+06,1.119429e+06
mean,1.399596e+05,1.399596e+05,2.020065e+05,6.256994e+05,3.192132e+04,1.957956e+01,1.109614e+00,9.092228e+05,1.493555e+06
std,8.084022e+04,8.084022e+04,1.112824e+00,4.965843e+06,3.125575e+05,8.062035e+01,6.542536e+00,1.321007e+04,1.194716e+04
min,1.000000e+00,1.000000e+00,2.020050e+05,1.000000e+00,-6.500000e+05,1.000000e+00,-1.000000e+00,8.756036e+05,1.458813e+06
25%,6.996500e+04,6.996500e+04,2.020060e+05,6.100000e+04,0.000000e+00,2.000000e+00,0.000000e+00,9.050022e+05,1.481856e+06
50%,1.399290e+05,1.399290e+05,2.020070e+05,1.853000e+05,0.000000e+00,5.000000e+00,0.000000e+00,9.093846e+05,1.499717e+06
75%,2.098930e+05,2.098930e+05,2.020080e+05,4.803600e+05,4.500000e+03,1.800000e+01,1.000000e+00,9.127210e+05,1.501364e+06
max,2.842650e+05,2.842650e+05,2.020080e+05,8.724686e+08,6.837222e+07,1.035900e+04,1.025000e+03,9.507241e+05,1.552940e+06


In [10]:
#data preprocessing

In [11]:
# geocoding
from pyproj import Proj, transform 
proj_itrf = Proj(init='epsg:5178') # korean grid
proj_wgs84 = Proj(init='epsg:4326') # World Geodetic System
df["lon"],df["lat"] = transform(proj_itrf,proj_wgs84,df['point_x'],df['point_y']) # itrf --> lon/lat


df['geometry'] = gpd.points_from_xy(df.point_x, df.point_y) # points 
df_new=df.drop(['point_x','point_y'],axis=1)
# df.drop(['point_x','point_y'],inplace=True, axis=1)

# get boundary
boundary = gpd.read_file(os.path.join(path, 'LSMD_ADM_SECT_UMD_50.shp'),encoding='euc-kr').to_crs('EPSG:4326')


In [12]:
boundary.head()

,EMD_CD,EMD_NM,SGG_OID,COL_ADM_SE,GID,geometry
0,50130310,안덕면,1698,50130,4976,"MULTIPOLYGON (((126.28206 33.30320, 126.28243 ..."
1,50130250,대정읍,1363,50130,4977,"MULTIPOLYGON (((126.16875 33.27659, 126.16876 ..."
2,50130116,강정동,2321,50130,4978,"MULTIPOLYGON (((126.46279 33.23248, 126.46292 ..."
3,50130106,서홍동,373,50130,4979,"MULTIPOLYGON (((126.51991 33.35191, 126.52008 ..."
4,50130101,서귀동,372,50130,4980,"MULTIPOLYGON (((126.55443 33.25228, 126.55445 ..."


In [13]:
df_new

,objectid,field1,ym,sido,sigungu,franclass,type,time,totalspent,disspent,numofspent,numofdisspent,lon,lat,geometry
0,1,1,202005,제주특별자치도,제주시,영세,일반한식,00시,363000,66500,10,2,126.176746,33.305090,POINT (877005.983 1479766.073)
1,2,2,202005,제주특별자치도,제주시,영세,단란주점,00시,1180000,0,3,0,126.176737,33.305540,POINT (877005.745 1479816.049)
2,3,3,202005,제주특별자치도,제주시,중소1,편의점,00시,157670,6850,20,2,126.177311,33.303746,POINT (877056.676 1479616.384)
3,4,4,202005,제주특별자치도,제주시,영세,편의점,00시,46600,0,2,0,126.177283,33.305098,POINT (877055.959 1479766.312)
4,5,5,202005,제주특별자치도,제주시,영세,주점,00시,66000,0,2,0,126.177264,33.305999,POINT (877055.482 1479866.263)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1119424,280081,280081,202008,제주특별자치도,제주시,영세,편의점,x시,13700,0,2,0,126.962369,33.515632,POINT (950266.598 1502457.688)
1119425,280082,280082,202008,제주특별자치도,제주시,영세,일반한식,x시,56000,0,1,0,126.962928,33.513833,POINT (950317.538 1502258.007)
1119426,280083,280083,202008,제주특별자치도,제주시,영세,일반한식,x시,28000,0,1,0,126.964580,33.510691,POINT (950469.158 1501908.867)
1119427,280084,280084,202008,제주특별자치도,제주시,중소1,중국음식,x시,46000,0,3,0,126.965779,33.500332,POINT (950574.639 1500759.806)


In [14]:
df_new.head()

,objectid,field1,ym,sido,sigungu,franclass,type,time,totalspent,disspent,numofspent,numofdisspent,lon,lat,geometry
0,1,1,202005,제주특별자치도,제주시,영세,일반한식,00시,363000,66500,10,2,126.176746,33.305090,POINT (877005.983 1479766.073)
1,2,2,202005,제주특별자치도,제주시,영세,단란주점,00시,1180000,0,3,0,126.176737,33.305540,POINT (877005.745 1479816.049)
2,3,3,202005,제주특별자치도,제주시,중소1,편의점,00시,157670,6850,20,2,126.177311,33.303746,POINT (877056.676 1479616.384)
3,4,4,202005,제주특별자치도,제주시,영세,편의점,00시,46600,0,2,0,126.177283,33.305098,POINT (877055.959 1479766.312)
4,5,5,202005,제주특별자치도,제주시,영세,주점,00시,66000,0,2,0,126.177264,33.305999,POINT (877055.482 1479866.263)


In [15]:
geo_df = gpd.GeoDataFrame(columns={'dong', 'geometry'})

In [16]:
# import sys
# from data import api_key 
# key=api_key.google_api_key

ModuleNotFoundError: ignored

In [ ]:
df=pd.concat(concat_list)

In [ ]:
from pyproj import Proj, transform 
proj_itrf = Proj(init='epsg:5178')
proj_wgs84 = Proj(init='epsg:4326')
df["lon"],df["lat"] = transform(proj_itrf,proj_wgs84,df['point_x'],df["point_y"])
df.drop(['point_x','point_y',"field1",'ym'],inplace=True, axis=1)

In [ ]:
# df['lon_lat']=df['lon'].astype(str)+'_'+df['lat'].astype(str)

In [ ]:
# df_lon_lat=df['lon_lat'].drop_duplicates()
# df_lon_lat

In [ ]:
# import googlemaps
# from datetime import datetime

# gmaps = googlemaps.Client(key) 
# reverse_geocode_result = gmaps.reverse_geocode((33.302315,126.176746))
# reverse_geocode_result
# # geocode_result  

In [ ]:
# import json
# m=folium.Map(location=[48,-102],zoom_start=3)
# folium.Choropleth(
#     geo_data='./data/03_us-states.json',
#     data=state_data, columns=["State",'Unemployment'],
#     nan_fill_color="purple", nan_fill_opacity=0.4,
#     key_on="    ", fill_color="YlGn",
#     legend_name="Unemployment Rate (%)"
# ).add_to(m)
# m

In [ ]:
#import rename module  
import type_names
input_data=type_names.rename()
for key, value in input_data:
    df["type"].replace(dict.fromkeys(key, value), inplace=True)

In [ ]:
df

In [ ]:
# data preprocessing

df['sigungu']=df['sigungu'].replace(['제주시','서귀포시'],['jeju','Seogwipo'])
df=df[df['time']!='x시'].reset_index(drop=True) # payments canceled 
df['time']=df['time'].apply(lambda x : x[:2])
df.franclass=df.franclass.replace(['영세', '중소1', '중소', '중소2', '일반'],['under_0.3m','under_1m','under_0.5m','under_3m','over_3m'])
df_final=df.rename(columns={'sigungu':'city','franclass':'class','totalspent':'total_income','disspent':'subsidies_income','numofspent':'num_of_payments','numofdisspent':'num_of_subspayment'})
df_final[['total_income','subsidies_income']]=df_final[['total_income','subsidies_income']].astype(int).apply(lambda x:x/1000) #won to $, 1000:1
df_final['non_subsidies_income']=df_final['total_income']-df_final['subsidies_income']

In [ ]:
df_final['month'].unique()

In [ ]:
# visualization 
#1. subsidies_income by month 
df_month_group=df_final.groupby('month')["subsidies_income"].sum().reset_index()

plt.figure(figsize=(10,6))
sns.catplot(x="month", y="subsidies_income", ci=None, kind="bar", data=df_month_group);

In [ ]:
df_final

In [ ]:
# all income by city 
# population = jeju_city (507,095), seogwipo (190,254), seogwipo/jeju_city = 0.375
df_city_group= df_final.groupby('city')['total_income','non_subsidies_income',"subsidies_income"].sum() 
df_city_group

f, ax = plt.subplots(1,3, figsize=(20,10))

df_city_group['total_income'].plot.pie(explode=[0,0.1], autopct='%1.1f%%',ax=ax[0], shadow=True,fontsize=20)
ax[0].set_title('total income by city',fontsize=20)

df_city_group['non_subsidies_income'].plot.pie(explode=[0,0.1], autopct='%1.1f%%',ax=ax[1], shadow=True,fontsize=20)
ax[1].set_title('non_subsidies_income by city',fontsize=20)

df_city_group['subsidies_income'].plot.pie(explode=[0,0.1], autopct='%1.1f%%',ax=ax[2], shadow=True, fontsize=20)
ax[2].set_title('subsidies_income by city',fontsize=20)


In [ ]:
#
df_5=df_final[df_final['month']=='5']
df_5_sum=df_5[['non_subsidies_income','subsidies_income']].sum()

plt.figure(figsize=(10,6))
df_5_sum_plot=df_5_sum.plot.pie(explode=[0,0.1], autopct='%1.1f%%', shadow=True,fontsize=20)


In [ ]:
df_5_group_type=df_5.groupby('type')['total_income','non_subsidies_income',"subsidies_income"].sum() 
df_5_group_type.iloc[1]

total_income            1.292806e+07
non_subsidies_income    1.091423e+07
subsidies_income        2.013829e+06
Name: automobile, dtype: float64

In [ ]:
#error fix
df_5_group_type.loc['beauty']=df_5_group_type.loc['beauty']+df_5_group_type.loc['화장품']
df_5_group_type.drop(index='화장품', inplace=True)

In [ ]:
df_5_group_type['replacement_rate']=df_5_group_type['subsidies_income']/df_5_group_type['non_subsidies_income']
df_5_group_type.sort_values('replacement_rate',ascending=False)

,total_income,non_subsidies_income,subsidies_income,replacement_rate
type,,,,
agriculture,6.499023e+06,4.669299e+06,1829724.194,0.391863
grocery,4.010482e+07,3.236285e+07,7741962.050,0.239224
housing,1.386279e+06,1.147461e+06,238817.841,0.208127
beauty,1.099585e+06,9.206987e+05,178886.697,0.194295
construction,4.322700e+06,3.621635e+06,701065.099,0.193577
automobile,1.292806e+07,1.091423e+07,2013829.169,0.184514
medical,1.110183e+07,9.426260e+06,1675572.420,0.177756
dinning,3.804076e+07,3.264970e+07,5391056.740,0.165118
service,4.397064e+06,3.777110e+06,619954.400,0.164135


In [22]:
import itertools
a=list(itertools.permutations('ABCD', 2))
print(a)

[('A', 'B'), ('A', 'C'), ('A', 'D'), ('B', 'A'), ('B', 'C'), ('B', 'D'), ('C', 'A'), ('C', 'B'), ('C', 'D'), ('D', 'A'), ('D', 'B'), ('D', 'C')]


In [29]:
df = gpd.read_file(gpd.datasets.get_path('nybb'))

In [30]:
df

,BoroCode,BoroName,Shape_Leng,Shape_Area,geometry
0,5,Staten Island,330470.010332,1.623820e+09,"MULTIPOLYGON (((970217.022 145643.332, 970227...."
1,4,Queens,896344.047763,3.045213e+09,"MULTIPOLYGON (((1029606.077 156073.814, 102957..."
2,3,Brooklyn,741080.523166,1.937479e+09,"MULTIPOLYGON (((1021176.479 151374.797, 102100..."
3,1,Manhattan,359299.096471,6.364715e+08,"MULTIPOLYGON (((981219.056 188655.316, 980940...."
4,2,Bronx,464392.991824,1.186925e+09,"MULTIPOLYGON (((1012821.806 229228.265, 101278..."
